### License
Copyright (C) 2021-2022, Xilinx, Inc.
<br>
Copyright (C) 2022-2024, Advanced Micro Devices, Inc.
<p>
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
<p>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS, 
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# ChipScoPy DDR Reporting Example


<img src="../img/api_overview.png" width="500" align="left">

## Description
This demo shows how to print and report DDR calibration status and report detailed information.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2024.1 installed and running
- Xilinx cs_server 2024.1 installed and running
- Python 3.8 or greater installed
- ChipScoPy 2024.1 installed
- Jupyter notebook support and extra libs needed - Please do so, using the command `pip install chipscopy[core-addons,jupyter]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- Paths to server(s) and files are set correctly

In [1]:
import pprint
import os
import json
from chipscopy import create_session, report_versions, delete_session
from chipscopy import get_design_files

In [2]:
# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# specify hw and if programming is desired
HW_PLATFORM = os.getenv("HW_PLATFORM", "vck190")
PROG_DEVICE = os.getenv("PROG_DEVICE", 'True').lower() in ('true', '1', 't')

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files(f"{HW_PLATFORM}/production/chipscopy_ced")

PDI_FILE = design_files.programming_file
LTX_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PDI_FILE}")
print(f"PROBES_FILE:{LTX_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: /scratch/chipscopy/venv/lib/python3.11/site-packages/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.pdi
PROBES_FILE:/scratch/chipscopy/venv/lib/python3.11/site-packages/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
# Start of the connection
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

                      ChipScoPy Version Information                      
┌────────────────────────────┬──────────────────────────────────────────┐
│                     Entity │ Version                                  │
├────────────────────────────┼──────────────────────────────────────────┤
│                  ChipScoPy │ ┌───────────┬──────────────────────┐     │
│                            │ │ Attribute │ Value                │     │
│                            │ ├───────────┼──────────────────────┤     │
│                            │ │     Build │ 2024.1.1717606426    │     │
│                            │ │ Timestamp │ Jun 05 2024-09:53:46 │     │
│                            │ └───────────┴──────────────────────┘     │
│                            │                                          │
│ hw_server @ localhost:3121 │ ┌───────────┬──────────────────────────┐ │
│                            │ │ Attribute │ Value                    │ │
│                            │ ├───────────┼──────────────────────────┤ │
│                            │ │    Status │ Connected                │ │
│                            │ │     Build │ 1716424495               │ │
│                            │ │   Version │ 2024.1                   │ │
│                            │ │ Timestamp │ Wed May 22 18:34:55 2024 │ │
│                            │ └───────────┴──────────────────────────┘ │
│                            │                                          │
│ cs_server @ localhost:3042 │ ┌───────────────┬──────────────────────┐ │
│                            │ │     Attribute │ Value                │ │
│                            │ ├───────────────┼──────────────────────┤ │
│                            │ │        Status │ Connected            │ │
│                            │ │         Build │ 1714182049           │ │
│                            │ │       Version │ 2024.1.0             │ │
│                            │ │     Timestamp │ Apr 26 2024-18:40:49 │ │
│                            │ │       Package │ 2024.1.1714182049    │ │
│                            │ │ Artifact type │ pyinstaller          │ │
│                            │ └───────────────┴──────────────────────┘ │
└────────────────────────────┴──────────────────────────────────────────┘

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
versal_device = session.devices.filter_by(family="versal").get()
if PROG_DEVICE:
    versal_device.program(PDI_FILE)
else:
    print("skipping programming")

--> INFO: Programming device with: 
/scratch/chipscopy/venv/lib/python3.11/site-packages/chipscopy/examples/designs/vck190/production/chipscopy_ced/chi
pscopy_wrapper.pdi

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use

In [5]:
versal_device.discover_and_setup_cores()
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


## 5 - Show enabled DDRs in the device. Pick one to use

In [6]:
ddr_list = versal_device.ddrs
for ddr in ddr_list:
    print(ddr, "  Enabled:", ddr.is_enabled)

# Grab the first enabled DDR
for ddr in ddr_list:
    if ddr.is_enabled:
        print("Using Enabled DDR: ", ddr)
        break

ddr_0   Enabled: True
ddr_1   Enabled: False
ddr_2   Enabled: False
ddr_3   Enabled: False
Using Enabled DDR:  ddr_0


## 6 - Getting the Calibration Status

There are several methods available to collect memory calibration information.

### Method 1 - Calibration PASS/FAIL status

In [7]:
# Method 1 - Use Status string base API directly
print(ddr, "calibration status:", ddr.get_cal_status())

ddr_0 calibration status: PASS


### Method 2 - Calibration from the status property group

In [8]:
# Use Property Group to get dictionary base of results
props = ddr.ddr_node.get_property_group(["status"])
print(pprint.pformat(props, indent=2))

{ 'cal_error_msg': 'None',
  'cal_message': 'No errors detected during calibration.',
  'cal_status': 'PASS',
  'dqs_status': 'Running',
  'health_status': 'GOOD',
  'track_enable': 'DQS Gate Tracking, UB ISR Tracking, DDRMC ISR Tracking, '
                  'Post-Cal Status'}


### Method 3 - Detailed calibration status for each stage

In [9]:
# Use get Cal Stages API directly to also get dictionary results
props = ddr.get_cal_stages()
print(pprint.pformat(sorted(props.items()), indent=2))

[ ('cal_stage.01_F0_PHY_BISC', 'Pass'),
  ('cal_stage.02_F0_MEM_INIT', 'Pass'),
  ('cal_stage.03_F0_DQS_GATE_CAL', 'Pass'),
  ('cal_stage.04_F0_WRITE_LEVELING', 'Pass'),
  ('cal_stage.05_F0_READ_DQ_CAL', 'Pass'),
  ('cal_stage.06_F0_WRITE_DQ_DBI_CAL', 'Pass'),
  ('cal_stage.07_F0_WRITE_LATENCY_CAL', 'Pass'),
  ('cal_stage.08_F0_READ_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.09_F0_WRITE_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.10_EN_VT_TRACK', 'Pass'),
  ('cal_stage.11_READ_DQS_TRACK', 'Pass'),
  ('cal_stage.12_CAL_DONE', 'Pass')]


## 7 - Generate Full DDRMC Report

The report() API call creates a full DDRMC status report to stdout or a file. This report includes memory configuration, margin analysis, calibration, and health status information.

In [10]:
# Use a single report command to get all latest essential
# Status and decoded data collected as it presents
ddr.report()
# Specify True to argument 1, and name/path to argument 2
# to get the report output generated and saved to a file
ddr.report(True, "test_out.txt")
print("Report Done.\n")

-------------------

DDRMC Status

-------------------

Calibration Status:   PASS

Overall Health:   GOOD

...

Byte  4   -  Left Margin:   143  ( 73 )  Center Point:   545  ( 278 )  Right Margin:   121  ( 62 )

Byte  5   -  Left Margin:   144  ( 74 )  Center Point:   572  ( 293 )  Right Margin:   121  ( 62 )

Byte  6   -  Left Margin:   144  ( 74 )  Center Point:   557  ( 285 )  Right Margin:   121  ( 62 )

Byte  7   -  Left Margin:   148  ( 76 )  Center Point:   623  ( 319 )  Right Margin:   128  ( 66 )

Report Done.



## 8 - Dump the complete set of internal properties as json

This demonstrates how to get a Python dictionary of all the low level DDR properties. These can be converted to JSON easily for export to other tools.

In [11]:
props = ddr.ddr_node.get_property_group([])
json_props = json.dumps(props, indent=4)
print(json_props)

{
    "cal_seq_status_24": 295,
    "f0_dqsgate_stg1_rldlyrnk_fine_rank0_byte2": 22,
    "f1_wrdqdbi_stg2_dq_odly_bit62": 0,
    "f1_rddq_pqtr_right_fcrse_nibble04": 0,
    "f0_rddq_nqtr_right_nibble06": 173,
    "f1_wrdqdbi_odly_dq_final_bit42": 0,
    "f0_rdcmplx_pqtr_right_short_nibble08": 166,
    "wrvref_vref_upp_bound_rank0_byte2": 0,
    "f0_wrlvl_offset_rank0_byte3": 82,
    "f0_rdcmplx_nqtr_right_nibble09": 142,
    "bisc_pqtr_align_nibble04": 0,
    "f1_rdcmplx_pqtr_right_short_fcrse_nibble01": 0,
    "bisc_idly0_align_nibble13": 76,
    "0x3d_READ_DQS_TRACK": "Pass",
    "f0_rddq_pqtr_right_nibble12": 170,
    "f0_wrdqdbi_odly_dq_final_bit22": 30,
    "pre_post_cal_map_ver": 1,
    "f0_wrdqdbi_stg3_dq_odly_bit04": 50,
    "f1_wrcmplx_odly_dq_final_bit56": 0,
    "f0_wrdqdbi_left_edge_dq_bit55": 38,
    "f0_rd_simp_fall_ctp_nib_07": [
        99,
        191
    ],
    "f1_wrprbs_right_margin_byte7": 0,
    "fifo_rden_dly": 19,
    "f0_wrdqdbi_stg3_dq_odly_bit42": 37,
    "f0

In [12]:
## When done with testing, close the connection
delete_session(session)